In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
plt.rc("figure", dpi=300, figsize=(9,3))
np.random.seed(2020)

<h1 align="center">Titanic Survival Estimation via Naïve Bayes</h1>
<h2 align="center">Angela Cao</h2>
<h5 align="center">Fall 2020 Directed Readings Program, University of Texas at Austin Department of Mathematics</h4>
<h5 align="center">Advised by Shane McQuarrie</h4>

## Contents

1. [**Introduction: The Titanic Problem**](#Introduction:-The-Titanic-Problem)
2. [**Brief Data Summary**](#Brief-Data-Summary)
3. [**The Naïve Bayes Algorithm**](#The-Naïve-Bayes-Algorithm)
4. [**Applying the Algorithm**](#Applying-the-Algorithm)

## Introduction: The Titanic Problem

On April 15, 1912, the [RMS Titanic](https://en.wikipedia.org/wiki/Titanic) sank, leaving about 1,500 people dead. The goal of this project is to construct a machine learning algorithm to predict whether or not a person with given characteristics would have been likely to survive the accident. This is a [popular problem](https://www.kaggle.com/c/titanic) for a first-time exposure to machine learning.

We have a [data set](https://www.kaggle.com/c/titanic/data) with information about 1,308 of the Titanic passengers, with the following features for each passenger. We begin by loading the data and displaying a few random entries.

In [ ]:
# Read the data from file.
titanic_original = pd.read_csv("titanic.csv")
titanic_original.sample(5)

Which of these features differentiate a survivor from a nonsurvivor? We restrict our attention to the following features:
- `Sex`: the sex of the passenger, either `male` or `female`.
- `Age`: the age of the passenger, usually recorded as a positive integer.
- `Pclass`: the class of the passenger's ticket, 1 for first class (the best), 2 for second class, and 3 for third class (the worst).
- `Fare`: the cost of the passenger's ticket.

Some of the other columns in the data may be useful, but certainly these features are more important survival indicators than, for example, the passenger `Name`.

In [ ]:
# Extract the relevant columns.
titanic = titanic_original[["Survived", "Sex", "Age", "Pclass", "Fare"]]
titanic.sample(5)

Before any analysis, we split the data into training and testing sets, reserving 15% of the entries for testing. We are only allowed to use the testing set to evaluate the performance of our algorithm later on. Since some of the entries of the data may be missing (for example, some passengers don't have a recorded age), we replace missing values of the test data with the column averages of the training data.

In [ ]:
# Split the data into training and testing sets.
train_data, test_data = train_test_split(titanic, test_size=.15)
print(f"{len(train_data)} train points, {len(test_data)} test points ({len(titanic)} total)")

# Separate the Survival labels from the test data and replace
# missing values with the averages from the training data.
test_labels = test_data["Survived"]
test_data = test_data.drop("Survived", axis=1).fillna(train_data.mean())

test_data.sample(3)

## Brief Data Summary

Before describing the algorithm, we give a brief statistical overview of the training data (remember, we aren't allowed to even look at the testing data). First, what was the rate of survival?

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9, 2))

survival_count = train_data.groupby("Survived").count().max(axis=1)
survival_count.plot(kind="barh", ax=ax)

ax.set_ylabel(""); ax.set_yticklabels(["perished", "survived"])
ax.set_xlabel("number of passengers")
ax.set_title(f"Passenger Survival Rate: {survival_count[1] / sum(survival_count):.2%}")
plt.show()

The bar graph above conveys the number of passengers in the data who survived or perished. From the bar graph above, more passengers, according to the data, perished, and the passenger survival rate is 38.13%. 

In [ ]:
fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(9,2))

train_data["Age"].plot(kind="hist", bins=80, ax=ax1)
ax1.set_xlabel("passenger age")

train_data["Fare"].plot(kind="hist", bins=40, ax=ax2)
ax2.set_ylabel(""); ax2.set_xlabel("ticket fare")
plt.show()

The histogram on the left shows the distribution of the ages of the passengers in the training data. The distribution is slightly right-skewed with a good amount of passengers in their 20's and 30's and also a good chunk of children as well.

The histogram on the right shows the distribution of the ticket prices of the passengers in the training data. This distribution is extremely right-skewed, with the overwhelming majority of passengers having tickets that cost \$100  or less.

Next, we briefly look for statistical differences between survivors and nonsurvivors.

In [ ]:
train_data.groupby(["Survived"]).mean()

This table sorts the data by the survival label (0 for non-survivors and 1 for survivors), then computes mean of the columns for each group. A few things are immediate apparent:
- Survivors had, on average, significantly more expensive tickets than non-survivors.
- Survivors, on average, were in a (slightly) higher class than non-survivors; non-survivors were highly likely to be in a lower or common class (like third-class). 
- Survivors, on average, were slightly younger than non-survirors. 

In short, the survivors and non-survivors exhibit different statistics; we aim to use that information to construct a "survival classifier."

## The Naïve Bayes Algorithm

Our goal is to construct a function that maps a given tuple of `Sex`, `Age`, `Pclass`, and `Fare` values to a `Survival` value (0 for perished or 1 for survived).
There are many ways to construct such a function, but we will use the simple Naïve Bayes algorithm.
This is a _binary classification problem_ because we are separating instances of data into two categories, and since we have labeled data to train on, this is an example of _supervised learning_. Naïve Bayes can also be used for multi-class classification (more than two categories to sort the data into), but we restrict ourselves to the binary case for ease of exposition. We will first develop the algorithm generally, then show how it applies specifically to the Titanic data.

#### The Algorithm

Let $P$ be a probability measure and let $A$ and $B$ be events in the probability space. _Bayes' rule_ is the statement

$$
P(A|B) = \frac{P(B|A)P(A)}{P(B)}.
$$

Suppose we have $n$ features (kinds of data to train on) $X_1,\ldots,X_n$ and let $Y$ denote the label corresponding to these features. We use Bayes' rule to write

$$
P(Y=i\mid X_1,\ldots,X_n) = \frac{P(X_1,\ldots,X_n\mid Y=i)P(Y=i)}{P(X_1,\ldots,X_n)}.
$$

The left-hand side is the probability that the tuple of data $(X_1,\ldots,X_n)$ belongs to the class $Y = i$.
The right-hand side has three elements:
- The probability $P(Y=i)$ is called the _prior_, which is our initial guess for the probability of an arbitrary set of data belonging to the class $Y = i$.
- The denominator $P(X_1,\ldots,X_n)$ is called the _evidence_, which is the normalization factor for the conditional probability.
- The conditional probability $P(X_1,\ldots,X_n \mid Y=i)$ is the _likelihood_, the probability of observing this specific data given that it belongs to the class $Y = i$. This is usually the trickiest part to compute in a Bayesian inference problem, but we will make an assumption that makes it easy to handle.

To deal with the evidence, we use with the _Law of Total Probability_:

$$
P(X_1,\ldots,X_n) = \sum_{i\in\{0,1\}} P(X_1,\ldots,X_n|Y=i)P(Y=i).
$$

In other words, the evidence can be written in terms of the likelihoods and priors. To compute the likelihood, we use the (naïve!) assumption that **the data features are conditionally independent**, that is,

$$
\begin{align*}
    P(X_1,\ldots,X_n|Y=i)
    &= \prod_{j=1}^{n}P(X_{j}|Y = i)
    \\
    &= P(X_1|Y=i)P(X_2|Y=i)\cdots P(X_n|Y=i)
\end{align*}
$$

**_Write a little bit here about why this may or may not be a good idea._**

Naïve Bayes might not always be a good algorithm to implement, because we're assuming that all the elements involved in the algorithm are independent to each other which might not always be the case (ex. there might be a correlation between fare (ticket) price and age or p-class). Also, the algorithm might not work for more statisically-complicated situations (one of them is if all the elements are not independent from each other) as well. We're oversimplying the situation where the algorithm is applied to and the math behind the situation. 

To compute each $P(X_j|Y=i)$, we need to choose probability distributions for each $X_j$. We'll give two examples.
1. If $X_1$ can take on two values 0 and 1 (e.g., in the titanic data, `Sex` is either `male` or `female`), then we choose a Bernoulli distribution to model the probability distribution of $X_1$, that is,
<!--  -->
$$
\begin{align*}
    P(X_1 = 1 \mid Y = i) &= q_i
    &
    P(X_1 = 0 \mid Y = i) &= 1 - q_i
\end{align*}
$$
<!--  -->
We estimate $q_i$ from the training data by counting the number of instances where $X_1 = 1$ among the group where $Y = i$. That is,
<!--  -->
$$
\begin{align*}
    q_i = \frac{\textrm{# data points with}\ X_1 = 1, Y = i}{\textrm{total # of data points with}\ Y = i}
\end{align*}
$$

2. For a continuous feature, we need to select and calibrate a continuous distribution. For example, we may assume $X_2$ has a Gaussian (Normal) distribution, defined by the mean $\mu$ and standard deviation $\sigma$. To estimate $\mu$ and $\sigma$ for each group, we again use the training data:
<!--  -->
$$
\begin{align*}
    \mu_{i} &= \textrm{mean}\left(\textrm{values of}\ X_2\ \textrm{where}\ Y = i\right)
    \\
    \sigma_{i} &= \textrm{standard deviation}\left(\textrm{values of}\ X_2\ \textrm{where}\ Y = i\right)
\end{align*}
$$
<!--  -->
We then compute $P(X_2 \mid Y = i)$ by evaluating the probability density function of the normal distribution $\mathcal{N}(\mu,\sigma)$ at the value of $X_2$.

Now we can compute $P(Y = i \mid X_1,\ldots,X_n)$ for any sample of data $X_1,\ldots,X_n$. To classify the data, we simply pick the most probable label.

$$
    \textrm{label of data}\ (X_1,\ldots,X_n)
    = \underset{i\in\{0,1\}}{\textrm{argmax}}\ P(Y = i \mid X_1,\ldots,X_n).
$$

#### Pros and Cons

**_Pros_**
- It's _scalable_, meaning it's cheap to do for really large data sets.
- It's highly interpretable: it gives you a probability for each class (so the uncertainty of the prediction is quantified) and you can look at each of the distributions it learns to understand its decision-making process.
- It's easy to implement, as we see below.
- It doesn't require any preprocessing like scaling.
- The algorithm works efficiently, and it could give a relatively good approximation. 
- Robust to outliers since we're looking things at the distribution sense. 

**_Cons_**
- For the algorithm to work effectively, all the variables must be independent to each other or we assume that they are independent to each other; will not work for a group of variables where several of the variables are interdependent on each other or the variables are not independent at all. 
- The algorithm will work on statisically simple situations (like if all variables are independent to each other for instance); will not work in situations where the relationship(s) is/are subtle or more statistically complicated situations. 
- Since the algorithm oversimplifies the statistical situation, accuracy is not always the best, especially in more interdependent or complicated situations.
- You need a lot of data for the algorithm to work or else the distribution will not be good. 

### Application to the Titanic Data

We want the probability of survival given the features in the data, so we want to compute

$$
P(\textrm{Survived} \mid \textrm{Sex,Age,Pclass,Fare})
= \frac{P(\textrm{Sex,Age,Pclass,Fare} \mid \textrm{Survived})P(\textrm{Survived})}{P(\textrm{Sex,Age,Pclass,Fare})}.
$$

To build a Naïve Bayes classifier, we need to choose distributions for each of the data features.
- $X_1$ = Sex. This is a binary variable, so we model it with a Bernoulli distribution.
- $X_2$
- $X_3$ Categorical (Bernoulli but with more than two possible outcomes)
- $X_4$ Exponential (or Gaussian)

**_Explain why we choose each of these (refer to the graphs in the data visualization section)_**

Sex and Pclass are discrete variables. Sex is binary as the passenger is either male or female, and since it's a binary variable with two options of male or female, it can be used with a Bernoulli distribution. While not binary, Pclass is a discrete variable nonetheless: the passenger is either first class (1) or second class (2) or third/common class (3), and thus, can be considered as categorical. Age and fare, which are factors in determing survivor/non-survivor, are continuous variables, but as we see from the brief data summary, they have different distributions: age can be approximated with a Normal (Gaussian) distribution, but fare looks exponential. These factors can also determine which of the passengers of either sex or class survive since a mix of both sexes or all classes survived and perished (ex. some 2nd class passengers survived and some didn't; some females survived and some didn't). 

#### Implementation

In [ ]:
class TitanicNaiveBayes:
    def fit(self, data):
        groups = data.groupby("Survived")
        
        sex_params = {}
        age_params = {}
        fare_params = {}
        class_params = {}
        
        for i in range(0, 2):
            group = groups.get_group(i)
            
            # Get the average number of males in the group
            num_males = len(group[group["Sex"] == "male"])
            num_females = len(group[group["Sex"] == "female"])
            total_group = len(group)
            male_ratio = num_males / total_group
            female_ratio = num_females / total_group
            sex_params[i] = (male_ratio, female_ratio)
            
            # Get the mean and standard deviation of the ages in the group
            age_mean = group["Age"].mean()
            age_std = group["Age"].std()
            age_params[i] = (age_mean, age_std)
            
            # Get the mean and standard deviation of the fare in the group
            loc, scale = stats.expon.fit(group["Fare"].dropna())
            fare_params[i] = (loc, scale)
            
            # Get the probability for each class in the group
            num_first = len(group[group["Pclass"] == 1])
            num_second = len(group[group["Pclass"] == 2])
            num_third = len(group[group["Pclass"] == 3])
            first_ratio = num_first / total_group
            second_ratio = num_second / total_group
            third_ratio = num_third / total_group
            class_params[i] = (first_ratio, second_ratio, third_ratio)
        
        self.sex_params_ = sex_params
        self.age_params_ = age_params
        self.fare_params_ = fare_params
        self.class_params_ = class_params
        return self
    
    def proba(self, data):
        results = []
        for i in range(0, len(data)):
            result = []
            for k in range(0, 2):
                
                p1 = 1.0
                male, female = self.sex_params_[k]
                if data.iloc[i, 0] == "male":
                    p1 = male
                else:
                    p1 = female
                
                mean, std = self.age_params_[k]
                p2 = stats.norm(mean, std).pdf(data.iloc[i, 1])
                
                p3 = 1.0
                first, second, third = self.class_params_[k]
                p_class = int(data.iloc[i, 2])
                if p_class == 1:
                    p3 = first
                elif p_class == 2:
                    p3 = second
                else:
                    p3 = third
                
                loc, scale = self.fare_params_[k]
                p4 = stats.expon(loc, scale).pdf(data.iloc[i, 3])
                
                result.append(np.exp(np.sum(np.log([p1, p2, p3, p4, .5]))))
            denom = sum(result)
            results.append(np.array(result) / denom)
        return np.array(results)
    def predict(self, data):
        """Predict a survival category for each row of data via Bayes' rule.

        Parameters
        ----------
        data : (m,4) pd.DataFrame
            Data to make a prediction for. Must have columns
            ["Sex", "Age", "Fare", "Pclass"].

        Returns
        -------
        labels : (m,) ndarray
            Perish / survival probabilities.
        """
        return np.argmax(self.proba(data), axis=-1)

    def accuracy(self, data, labels):
        """Compute the accuracy of the model given some test data.

        Parameters
        ----------
        data : (m,4) pd.DataFrame
            Data to make a prediction for. Must have columns
            ["Sex", "Age", "Fare", "Pclass"].

        Returns
        -------
        accuracy : float
            The % of rows guessed correctly.
        """
        return np.mean(self.predict(data) == labels)
    
    # Distribution plots ------------------------------------------------
    def _survival_label(self, Y):
        return "Perished" if Y == 0 else "Survived"

    def plot_prior(self):
        data = pd.DataFrame(index=["Perished", "Survived"],
                            columns=["Prior"])
        data["Prior"] = self.prior_.pmf([0,1])
        data.plot(kind="barh")
        plt.legend([])
        plt.title("Prior Survival Distribution")

    def plot_sex(self):
        sexes = np.array([0, 1])
        data = pd.DataFrame(index=["Perished", "Survived"],
                            columns=["Female", "Male"])
        for k in range(0, 2):
            male, female = self.sex_params_[k]
            data.iloc[k, 0] = female
            data.iloc[k, 1] = male
        data.plot(kind="barh")
        plt.legend(loc="upper right")
        plt.title("Estimated Sex Distributions")

    def plot_age(self):
        years = np.linspace(0, 100, 1000)
        for Y, distributions in self.likelihood_.items():
            plt.plot(years, distributions["Age"].pdf(years),
                     label=self._survival_label(Y))
        plt.legend(loc="upper right")
        plt.title("Estimated Age Distributions")

    def plot_fare(self):
        fares = np.linspace(0, 550, 1000)
        for Y, distributions in self.likelihood_.items():
            plt.plot(fares, distributions["Fare"].pdf(fares),
                     label=self._survival_label(Y))
        plt.legend(loc="upper right")
        plt.title("Estimated Fare Distributions")

    def plot_pclass(self):
        data = pd.DataFrame(index=["Perished", "Survived"],
                            columns=["1st Class", "2nd Class", "3rd Class"])
        for Y, distributions in self.likelihood_.items():
            data.loc[self._survival_label(Y)] = distributions["Pclass"]
        data.plot(kind="barh")
        plt.legend(loc="upper right")
        plt.title("Estimated Pclass Distributions")

    def plot_likelihoods(self):
        """Plot each of the learned distributions."""
        self.plot_prior()
        plt.show()
        self.plot_sex()
        plt.show()
        self.plot_age()
        plt.show()
        self.plot_fare()
        plt.show()
        self.plot_pclass()
        plt.show()

In [ ]:
class TitanicNaiveBayes2:
    """Naïve Bayes classifier for the Titanic problem, mapping values of
    Sex, Age, Fare, and Pclass to Survival (0 = perished, 1 = survived).

    Attributes
    ----------
    prior_ : scipy.stats Bernoulli distribution
        Survival probability prior, i.e., prior_.pmf(Y) returns the
        probability of perishing if Y = 0 or surviving if Y = 1.

    likelihood_ : dict[Y -> scipy.stats distributions]
        Independent likelihood probability distributions whose parameters
        are computed from the training data. In order:
        * Sex: Bernoulli
        * Age: Gaussian (normal)
        * Fare: Exponential
        * Pclass: Categorical
    """
    def __init__(self, survival_prior=.5):
        """Set the probability of survival (Benoulli prior distribution)."""
        self.prior_ = stats.bernoulli(survival_prior)

    def fit(self, data):
        """Calculate statistics for each group based on survival.

        Parameters
        ----------
        data : pd.DataFrame
            Data to train on. Must have columns
            ["Survived", "Sex", "Age", "Fare", "Pclass"].

        Returns
        -------
        self
        """
        groups = data.groupby("Survived")
        self.likelihood_ = {}

        for Y, group in groups:
            # Record the label.
            distributions = {}

            # Sex distribution: Bernoulli.
            q = np.mean(group["Sex"] == "male")
            distributions["Sex"] = stats.bernoulli(q)

            # Age distribution: Gaussian (Normal).
            µ, σ = group["Age"].mean(), group["Age"].std()
            distributions["Age"] = stats.norm(µ, σ)

            # Fare distribution: Exponential.
            loc, scale = stats.expon.fit(group["Fare"].dropna())
            distributions["Fare"] = stats.expon(loc, scale)

            # Pclass distribution: Categorical.
            qs = np.array([np.mean(group["Pclass"] == j) for j in [1,2,3]])
            distributions["Pclass"] = qs

            self.likelihood_[Y] = distributions

        return self

    def proba(self, data):
        """Calculate the probabilites of each row of data belonging to each
        survival category via Bayes' rule:

                                        P(features | Survived=i) P(Survived)
        P(Survived=i | features) = ---------------------------------------------
                                   sum_{j}[P(features | Survived=j) P(Survived)]

        Parameters
        ----------
        data : (m,4) pd.DataFrame
            Data to make a prediction for. Must have columns
            ["Sex", "Age", "Fare", "Pclass"].

        Returns
        -------
        probabilities : (m,2) ndarray
            Perish / survival probabilities.
        """
        numerators = []
        for Y in self.likelihood_:
            dist = self.likelihood_[Y]

            # Evaluate the likelihood and prior distributions.
            pXs = [dist["Sex"].pmf(data["Sex"] == "male"),        # P(Sex|Y)
                   dist["Age"].pdf(data["Age"]),                  # P(Age|Y)
                   dist["Fare"].pdf(data["Fare"]),                # P(Fare|Y)
                   dist["Pclass"][np.int32(data["Pclass"]) - 1],  # P(Pclass|Y)
                   self.prior_.pmf([Y]*len(data))                 # P(Y)
                   ]

            numerators.append(np.exp(np.sum(np.log(pXs), axis=0)))
        numers = np.array(numerators)

        return np.transpose(numers / np.sum(numers, axis=0))

    def predict(self, data):
        """Predict a survival category for each row of data via Bayes' rule.

        Parameters
        ----------
        data : (m,4) pd.DataFrame
            Data to make a prediction for. Must have columns
            ["Sex", "Age", "Fare", "Pclass"].

        Returns
        -------
        labels : (m,) ndarray
            Perish / survival probabilities.
        """
        return np.argmax(self.proba(data), axis=-1)

    def accuracy(self, data, labels):
        """Compute the accuracy of the model given some test data.

        Parameters
        ----------
        data : (m,4) pd.DataFrame
            Data to make a prediction for. Must have columns
            ["Sex", "Age", "Fare", "Pclass"].

        Returns
        -------
        accuracy : float
            The % of rows guessed correctly.
        """
        return np.mean(self.predict(data) == labels)

    # Distribution plots ------------------------------------------------
    def _survival_label(self, Y):
        return "Perished" if Y == 0 else "Survived"

    def plot_prior(self):
        data = pd.DataFrame(index=["Perished", "Survived"],
                            columns=["Prior"])
        data["Prior"] = self.prior_.pmf([0,1])
        data.plot(kind="barh")
        plt.legend([])
        plt.title("Prior Survival Distribution")

    def plot_sex(self):
        sexes = np.array([0, 1])
        data = pd.DataFrame(index=["Perished", "Survived"],
                            columns=["Female", "Male"])
        for Y, distributions in self.likelihood_.items():
            data.loc[self._survival_label(Y)] = distributions["Sex"].pmf(sexes)
        data.plot(kind="barh")
        plt.legend(loc="upper right")
        plt.title("Estimated Sex Distributions")

    def plot_age(self):
        years = np.linspace(0, 100, 1000)
        for Y, distributions in self.likelihood_.items():
            plt.plot(years, distributions["Age"].pdf(years),
                     label=self._survival_label(Y))
        plt.legend(loc="upper right")
        plt.title("Estimated Age Distributions")

    def plot_fare(self):
        fares = np.linspace(0, 550, 1000)
        for Y, distributions in self.likelihood_.items():
            plt.plot(fares, distributions["Fare"].pdf(fares),
                     label=self._survival_label(Y))
        plt.legend(loc="upper right")
        plt.title("Estimated Fare Distributions")

    def plot_pclass(self):
        data = pd.DataFrame(index=["Perished", "Survived"],
                            columns=["1st Class", "2nd Class", "3rd Class"])
        for Y, distributions in self.likelihood_.items():
            data.loc[self._survival_label(Y)] = distributions["Pclass"]
        data.plot(kind="barh")
        plt.legend(loc="upper right")
        plt.title("Estimated Pclass Distributions")

    def plot_likelihoods(self):
        """Plot each of the learned distributions."""
        self.plot_prior()
        plt.show()
        self.plot_sex()
        plt.show()
        self.plot_age()
        plt.show()
        self.plot_fare()
        plt.show()
        self.plot_pclass()
        plt.show()

***Above are three different implementations of the Naive Bayes Algorithm in terms of the Titanic data.***

## Applying the Algorithm

Before we use Naïve Bayes on the Titanic data, we should check the assumption of conditional independence among the training variables.

In [ ]:
# Are these variables correlated?
train_data.drop("Survived", axis=1).corr()

**_This is why we \*might\* have a chance with Naïve Bayes, it looks like the features aren't too interdependent._**

In [ ]:
nb = TitanicNaiveBayes().fit(train_data)
nb.plot_sex()

In [ ]:
nb.proba(test_data)

In [ ]:
nb.predict(test_data)

In [ ]:
nb.accuracy(test_data, test_labels)

In [ ]:
samples = test_data.sample(5)
samples

In [ ]:
nb.proba(samples)

In [ ]:
nb.predict(samples)

In [ ]:
test_labels[samples.index]

In [ ]:
test_data.iloc[np.argmax(nb.proba(test_data), axis=0)]

## Conclusion

In this presentation, we gave an overview of the Naive Bayes algorithm, and implemented the algorithm for the Titanic problem. In the data, we decided to use a limited amount of columns or factors to determine the While Naive Bayes is not a sophisticated algorithm to use for the Titanic problem. However, we can do better with the same amount of data. 

## Appendix: More Data Visualizations

In [ ]:
titanic.groupby("Pclass").count()["Age"].plot(kind="barh")
plt.title("Passenger Count by Ticket Class")
plt.show()

The bar graph above shows the number of passengers in the data that are in each class based on the ticket. From the bar graph, the majority of passengers, recorded in the data, are in 3rd class followed by 1st and 2nd class by a relatively small gap between the two higher classes. 

In [ ]:
train_data.describe()

Not only shows the amount of passengers recorded in the data through the 'Survived' column (1309 passengers recorded, and most of them probably perished according to the mean and quartiles), but also the number of passengers in the data who got their ages recorded and the mean and ranges of ages recorded, and the mean and ranges of classes and fares recorded as well through min, quartiles, and max. 

From the table, we can see that: 
- The majority of passengers recorded perished in the Titanic incident. 
- There were 1046 passengers whose ages were recorded in the data. Some passengers are missing ages. 
- The mean age of the passsengers in the Titanic were 29-30 years old. 
- The median age of the passengers in the Titanic is about 28 years old which confirms the right-skewness of the distribution of ages. 
- The youngest passenger was a baby while the oldest passenger is 80 years old. 
- A good majority of passengers recorded bought 3rd class tickets though the mean (average) was is about 2nd class. 
- The mean fare of the passengers in the Titanic was about 33.30 dollars. 
- The median fare of the passengers in the Titanic was about 14.45 dollars which confirms the extreme right-skewness of the distribution of fare. 
- The cheapest ticket was for free while the most expensive ticket was about 512.33 dollars. 

**Missing data**

In [ ]:
titanic.isnull().sum()

The table above shows all the passengers in the data whose ages are unknown so far. According to the table, there are 263 entries or passengers whose ages were not recorded and one passenger with a missing ticket fare.

**Survived Classification Data Summary**

In [ ]:
survival_groups = titanic.groupby("Survived")
survival_groups.boxplot(grid=False, column=['Age'], layout=(1,2), vert=False)
survival_groups.boxplot(grid=False, column=['Fare'], layout=(1,2), vert=False)
plt.show()

The box plots above show the distributions of age and fare based on survival/non-survival. While it may seem that survivors may be slightly older and bought slightly more expensive tickets on average compared to their non-survivor counterparts, there are a lot of outliers all the boxplots.

**Pclass Classification Data Summary**

In [ ]:
groups = titanic.groupby("Pclass")
groups.boxplot(grid=False, column=['Age'], layout=(1,3))
groups.boxplot(grid=False, column=['Fare'], layout=(1,3))
plt.show()

The box plots above show the distributions of age and fare based on Pclass (1, 2, 3). 

There are no outliers in the distribution of age in the 1st class, but there are many outliers in the other classes in terms of age, especially 3rd class passengers since there is more data collected for 3rd class compared to the 2 higher classes. The average age of 1st class passengers is older than the other two classes, and the average age decreases as we get lower in class. 

There are outliers in the distributions of fare for all classes, especially 1st class since we expect that more expensive tickets are brought through this class. There are a lot of outliers for the distribution of fare for 3rd class passengers not just only because we expect the cheapest tickets from this class, but also how there is more data collected from this class as well. The average fare of 1st class passengers is obviously higher/more expensive than the other two classes, and the average fare decreases as we get lower in class. 

**Sex Classification Data Summary**

In [ ]:
gender_groups = titanic.groupby("Sex")
gender_groups.boxplot(grid=False, column=['Age'], layout=(1,2), vert=False)
gender_groups.boxplot(grid=False, column=['Fare'], layout=(1,2), vert=False)
plt.show()

The box plots above show the distributions of age and fare based on sex (male/female). There is only one outlier in the age distribution of females while there are multiple outliers in the age distribution of males. The average age of males and females is about the same, but with average age of males slightly higher. Then, there are a lot of outliers in the female and male distributions of fare. The average fare of males is slightly lower than females. Since passengers of cheaper tickets or older age are more likely to perish in the Titanic, it could be possible that sex played a significant role in who gets to survive and who doesn't in the Titanic. 

**Age and Fare as Classification**

In [ ]:
# Partition the passengers into 3 categories based on age.
age = pd.cut(titanic['Age'], [0, 12, 18, 80])
age

titanic.pivot_table(values='Survived', index=['Sex', age],
                   columns='Pclass', aggfunc='mean')

This is a two-way table of survival rate between sex and then age divided into intervals (children (0-12), teen (12-18), and adult (18-80)), and Pclass (1, 2, 3). (Sex + Age vs PClass / PClass vs Sex + Age)

Groups with everyone surviving: 
- 2nd class female children
- 1st class female teenagers
- 1st class male children
- 2nd class male children

In [ ]:
titanic.pivot_table(values='Survived', index=['Sex', age],
                   columns='Pclass', aggfunc='count')

This is a two-way table of number of survivors between sex and then age divided into intervals (children (0-12), teen (12-18), and adult (18-80)), and Pclass (1, 2, 3). (Sex + Age vs PClass / PClass vs Sex + Age)

In [ ]:
# Cut the ticket price into two intervals (cheap vs expensive)
fare = pd.qcut(titanic['Fare'], 2)
fare

titanic.pivot_table(values='Survived',
                   index=['Sex', age], columns=[fare, 'Pclass'],
                   aggfunc='count', fill_value='-')

This is a two-way table of number of survivors between sex and then age divided into intervals (children (0-12), teen (12-18), and adult (18-80)), and fare divided into intervals (cheap (0-14.454) and expensive-ish (14.454, 512.329)) and then Pclass (1, 2, 3). (Sex + Age vs Fare + PClass / Fare + PClass vs Sex + Age)

**Age and Fare Correlation**

In [ ]:
titanic.plot(kind='scatter', x='Age', y='Fare', alpha=.2)

There is a weak correlation between age and fare based on the graph above, so a correlation between the two variables is not clear. 